# Home Credit Modeling

In [65]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr, randint, uniform
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_objective
import pickle
import gc

%load_ext autotime

gc.enable()
np.random.seed(235)

path = "/Users/dsaxton/home_credit_default/"

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.06 ms


#### Load the data

In [85]:
df = pd.read_csv(path + "preprocessed_train.csv")

time: 26.9 s


In [86]:
sk_id_curr = df.pop("SK_ID_CURR")
y = df.pop("TARGET")

time: 887 ms


#### Select features

In [87]:
clf = lgb.LGBMClassifier(n_estimators=1000, num_leaves=23, subsample=0.5)
clf.fit(df, y)

lgb_cols = df.columns[clf.feature_importances_ > 0]
len(lgb_cols)

480

time: 2min 14s


In [88]:
var_imp = pd.DataFrame({"Feature": df.columns, "Importance": clf.feature_importances_})[["Feature", "Importance"]].sort_values("Importance", ascending=False)

time: 4.96 ms


#### View important features

In [95]:
var_imp.head(50)

,Feature,Importance
31,EXT_SOURCE_3,391
30,EXT_SOURCE_2,359
339,AMT_CREDIT_DIV_AMT_ANNUITY,305
375,EXT_SOURCE_1_DIV_DAYS_BIRTH,228
8,DAYS_BIRTH,220
363,DAYS_REGISTRATION_DIV_DAYS_LAST_PHONE_CHANGE,205
9,DAYS_EMPLOYED,180
342,EXT_SOURCE_PROD,174
11,DAYS_ID_PUBLISH,172
333,TOTAL_AMT_CREDIT_SUM_POS_DAYS_DIV_SUM_DAYS_CRE...,170


time: 12.2 ms


#### Average importance over non-zero importance features

In [90]:
int(np.nanmean(var_imp["Importance"].where(var_imp["Importance"] > 0)))

45

time: 6.15 ms


#### Importance of selected features

In [91]:
features = []

var_imp[var_imp["Feature"].isin(features)]

,Feature,Importance


time: 6.43 ms


#### Features with zero importance

In [92]:
var_imp.query("Importance == 0")

,Feature,Importance
85,FLAG_DOCUMENT_7,0
411,NAME_FAMILY_STATUS_Unknown,0
471,ORGANIZATION_TYPE_Industry:_type_6,0
511,FONDKAPREMONT_MODE_nan,0
505,ORGANIZATION_TYPE_XNA,0
504,ORGANIZATION_TYPE_University,0
88,FLAG_DOCUMENT_10,0
506,ORGANIZATION_TYPE_nan,0
82,FLAG_DOCUMENT_4,0
512,HOUSETYPE_MODE_block_of_flats,0


time: 12.9 ms


#### LightGBM CV

In [ ]:
params = {"n_estimators": 20000, 
          "num_leaves": 113, 
          "max_depth": 13,
          "learning_rate": 0.003, 
          "subsample": 0.5, 
          "colsample_bytree": 0.5, 
          "reg_lambda": 0.8}

lgb_data = lgb.Dataset(data=df[lgb_cols], 
                       label=y)

cv_result = lgb.cv(params=params, 
                   train_set=lgb_data, 
                   nfold=5, 
                   metrics="auc", 
                   early_stopping_rounds=200, 
                   stratified=True, 
                   shuffle=True, 
                   verbose_eval=100, 
                   show_stdv=True, 
                   seed=123)

cv_result = pd.DataFrame(cv_result)

/Users/dsaxton/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's auc: 0.759285 + 0.00299913
[200]	cv_agg's auc: 0.762315 + 0.00295487
[300]	cv_agg's auc: 0.764759 + 0.00284802
[400]	cv_agg's auc: 0.766687 + 0.00286511
[500]	cv_agg's auc: 0.768483 + 0.00276088
[600]	cv_agg's auc: 0.770183 + 0.00261199
[700]	cv_agg's auc: 0.771775 + 0.00256216
[800]	cv_agg's auc: 0.773383 + 0.00249986
[900]	cv_agg's auc: 0.774775 + 0.00243562
[1000]	cv_agg's auc: 0.775892 + 0.00241433
[1100]	cv_agg's auc: 0.777092 + 0.00228844
[1200]	cv_agg's auc: 0.778086 + 0.00225557
[1300]	cv_agg's auc: 0.779087 + 0.00220852
[1400]	cv_agg's auc: 0.779953 + 0.00217685
[1500]	cv_agg's auc: 0.780741 + 0.00213519
[1600]	cv_agg's auc: 0.781505 + 0.0020762
[1700]	cv_agg's auc: 0.782161 + 0.00203971
[1800]	cv_agg's auc: 0.78279 + 0.00200276
[1900]	cv_agg's auc: 0.783375 + 0.00199682
[2000]	cv_agg's auc: 0.783941 + 0.00196246
[2100]	cv_agg's auc: 0.784435 + 0.00197084
[2200]	cv_agg's auc: 0.784918 + 0.00197256
[2300]	cv_agg's auc: 0.785375 + 0.00197646
[2400]	cv_agg's auc: 0

In [ ]:
cv_result.tail()

#### Fit chosen model

CV mean:  0.790438

In [78]:
lgb_model = lgb.LGBMClassifier(n_estimators=6876, 
                         num_leaves=113, 
                         max_depth=10,
                         learning_rate=0.0025, 
                         subsample=0.5, 
                         colsample_bytree=0.5, 
                         reg_lambda=0.8)

lgb_model.fit(df[lgb_cols], y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
        learning_rate=0.0025, max_depth=10, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=6876,
        n_jobs=-1, num_leaves=113, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.8, silent=True, subsample=0.5,
        subsample_for_bin=200000, subsample_freq=1)

time: 30min 31s


#### Score test data and generate submission

In [79]:
df = pd.read_csv(path + "preprocessed_test.csv")

time: 4.26 s


In [80]:
df["NAME_INCOME_TYPE_Maternity_leave"] = 0

time: 10 ms


In [81]:
submission = pd.DataFrame({"SK_ID_CURR": df["SK_ID_CURR"], 
                           "TARGET": lgb_model.predict_proba(df[lgb_cols])[:,1]})

submission.head()

,SK_ID_CURR,TARGET
0,100001,0.033505
1,100005,0.123829
2,100013,0.026391
3,100028,0.033826
4,100038,0.170974


time: 26.3 s


In [82]:
submission.to_csv(path + "submission.csv", index=False)

time: 176 ms
